# AI for BI: Analysing Customer Feedback with AI


## Using synthetic data we analyse the customer feedback from the sales of shoes from a fictional online show shop. 

We initially read the Markdown formatted data, format it as as JSON data and save it in a file for two reasons - first, so we can separate two crews (one for creating the JSON and the other for writing a report) wothout having to clog up the context wondow of the second one with the output of the first, and second we can use the file to create charts in a hybrid application.


In [1]:
from crewai import Agent, Task, Crew
llm = "gpt-4o-mini"

Import the tools required to read and write files

In [2]:
from crewai_tools import tool
from crewai_tools import FileReadTool, FileWriterTool

# Initialize the tool to read any files the agents knows or lean the path for
file_read_tool = FileReadTool()
file_writer_tool = FileWriterTool()

In [3]:
# Define agent

fb_analyser_agent = Agent(
        role="Feedback analyser",
        goal="""Read the file provided it is in Markdown format. 
                It contains data about the range of items in an online shoe shop.
                Following this is a set of messages from customers giving feedback about the shoes that they have purchased.
                You goal is to:
                1. Calculate the sentiment for every customer message
                2. Create a structured file in JSON format that records a list of all customer feedback messages. 
                   Each item in the list should be structured as follows (the items in curly brackets should be calculated or extracted from the data):
                    {"Shoe": "{Name of item}", 
                     "Overall Rating": "{rating as given by customer}", 
                     "Issue": "{any issues identified - 'None' if none identified}", 
                     "Review": "{customer message}", 
                     "Sentiment": "{calculated sentiment of customer message}"}        
                3. Save the file using the name provided.
           """,
        backstory="You aim is to read and analyse customer feedback for items sold online",
        tools=[file_read_tool, file_writer_tool],
        llm=llm,
        allow_code_execution=False
    )

quality_analyser_agent = Agent(
        role="Quality analyser",
        goal="""Using the JSON structured customer feedback, write a report that consolidates 
                and summarizes the customer feedback.                 
                The report should be in Markdown format.
                Save the report in the file './report.md'. 
                
                The report should be structured in the
                following way (text in curly brackets should be calculated or otherwise derived from the data; 
                text in square backets are instructions that you should follow):

                     # Product review report

                     ### Summary

                     [create a table with an entry for each product in the follow format]
                     
                     | Product | Average Rating | Number of reviews | Positve | Neutral | Negative |
                     | {product name} | {the average of all the rating for this product} | {total number of reviews} | {number of positive reviews} | {number of neutral reviews} | {number of negative reviews} |

                     ### Insights

                     #### Best performers

                     {report on the products with the best reviews}

                     #### Underperformers

                     {report on the products that are underperforming}

                     #### Issues

                     {what steps need to be taken to improve sales}
                   

           """,
        backstory="You aim is to read and analyse customer feedback for items sold online",
        tools=[file_read_tool, file_writer_tool],
        llm=llm,
        allow_code_execution=False
    )



In [6]:
create_JSON = Task(
    description="""Analyse "shoes range and reviews.md" and save it in "./fb.json".'
                    """,
    expected_output="""A JSON file""",
    agent=fb_analyser_agent,
    tools=[file_read_tool, file_writer_tool]
)

create_report = Task(
    description="""Using the data in the file './fb.json', create a summary report forand save it in "./report.md".'
                    """,
    expected_output="""A Markdown file""",
    agent=quality_analyser_agent,
    tools=[file_read_tool, file_writer_tool]
)

# Define the crew
crew = Crew(
    agents=[fb_analyser_agent],
    tasks=[create_JSON],
    verbose=True
)
#crew.kickoff()



2024-11-26 19:09:33,712 - 17320 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [8]:
crew = Crew(
    agents=[quality_analyser_agent],
    tasks=[create_report],
    verbose=True
)
crew.kickoff()

2024-11-26 19:09:58,030 - 17320 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Quality analyser
## Task: Using the data in the file './fb.json', create a summary report forand save it in "./report.md".'
                    


# Agent: Quality analyser
## Thought: I need to read the content of the JSON file to analyze customer feedback for products sold online.
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"./fb.json\"}"
## Tool Output: 
[{"Shoe": "CloudStrider Sneakers", "Overall Rating": "2/5", "Issue": "Durability", "Review": "The shoes were super comfortable initially, but after just two months, the sole started to peel off! Really disappointed with the durability.", "Sentiment": "Negative"},{"Shoe": "CloudStrider Sneakers", "Overall Rating": "1/5", "Issue": "Sizing", "Review": "I ordered my usual size, but these sneakers were way too tight. Had to return them. The sizing is totally off.", "Sentiment": "Negative"},{"Shoe": "CloudStrider Sneakers", "Overall Rating": "3/5", "Issue": "Odor", "Review": "Great for short runs, but t

CrewOutput(raw='```\n# Product review report\n\n### Summary\n\n| Product                 | Average Rating | Number of reviews | Positive | Neutral | Negative |\n|-------------------------|----------------|-------------------|----------|---------|----------|\n| CloudStrider Sneakers    | 2.75           | 4                 | 1        | 1       | 2        |\n| PeakTrek Hikers         | 3.00           | 4                 | 2        | 1       | 1        |\n| SwiftShine Heels        | 2.50           | 4                 | 1        | 1       | 2        |\n| LuxeLace Oxfords        | 4.50           | 4                 | 3        | 1       | 0        |\n| EcoStep Slip-Ons        | 4.50           | 4                 | 3        | 1       | 0        |\n| BounceMax Runners       | 4.75           | 4                 | 4        | 0       | 0        |\n\n### Insights\n\n#### Best performers\n- **BounceMax Runners**: With an exceptional average rating of 4.75, these shoes received positive feedback acro

make two apps one to generate the files and a streamlit one to report with charts

Make a hybrid app by including conventional coding - create a Strreamlit app from the code above, show the report in one pane and the following charts in another.

In [9]:
import pandas as pd

df = pd.read_json("fb.json")
df

,Shoe,Overall Rating,Issue,Review,Sentiment
0,CloudStrider Sneakers,2/5,Durability,"The shoes were super comfortable initially, bu...",Negative
1,CloudStrider Sneakers,1/5,Sizing,"I ordered my usual size, but these sneakers we...",Negative
2,CloudStrider Sneakers,3/5,Odor,"Great for short runs, but they hold onto odors...",Neutral
3,CloudStrider Sneakers,5/5,None,Extremely lightweight and breathable. Perfect ...,Positive
4,PeakTrek Hikers,1/5,Waterproof,"These boots claim to be waterproof, but my fee...",Negative
5,PeakTrek Hikers,2/5,Grip,I slipped multiple times on rocky trails becau...,Negative
6,PeakTrek Hikers,4/5,Padding,The boots are sturdy and provide decent ankle ...,Neutral
7,PeakTrek Hikers,5/5,None,Used these for a weekend hike in the mountains...,Positive
8,SwiftShine Heels,2/5,Durability,"I wore these to a wedding, and the heel broke ...",Negative
9,SwiftShine Heels,3/5,Scratching,"While they look stunning, the glossy finish sc...",Neutral


In [29]:
import plotly.express as px

px.histogram(df,x="Sentiment", color='Shoe')

In [ ]:
fig = px.histogram(df,x="Overall Rating", color='Shoe')
fig.update_xaxes(categoryorder='array',
                 categoryarray= ['1/5','2/5','3/5','4/5','5/5'])

![](../images/Screenshot-fb-reporter-app.png)

![](image.png)